In [1]:
import itertools
import os
import sys
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage.io

from collections import defaultdict
from tqdm.auto import tqdm
from joblib import Parallel, delayed
import re
import h5py
import napari
import seaborn as sns

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
p_dir = (Path().cwd().parents[0]).absolute()

module_path = str(p_dir / "src")

if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
data_dir = (Path().cwd().parents[0] / 'data').absolute()

In [5]:
# import PPIGraph

# Read per cell data

In [23]:
# Read PPI dfs
PPI_save_path =  data_dir / '9PPI' / 'PPI'

dfs = []
for path in os.listdir(PPI_save_path):
    if 'csv' in path:
        df = pd.read_csv(PPI_save_path / path)
        dfs.append(df)

df = pd.concat(dfs)

# Group by location (Cell vs Nuclei)
g = df.groupby(['Condition', 'FOV', 'PPI', 'Cyto']).size()
df_cell = pd.DataFrame({'Count Cyto': g}).reset_index()
df_cell = df_cell[df_cell.Cyto != 0]
df_cell.columns = ['Condition', 'FOV', 'PPI', 'Id', 'Count_cell']

g = df.groupby(['Condition', 'FOV', 'PPI', 'Nuclei']).size()
df_nuclei = pd.DataFrame({'Count Nuclei': g}).reset_index()
df_nuclei= df_nuclei[df_nuclei.Nuclei != 0]
df_nuclei.columns = ['Condition', 'FOV', 'PPI', 'Id', 'Count_nuclei']

# Concat 
df_all = df_cell.merge(df_nuclei, how='left', on=['Condition', 'FOV', 'PPI', 'Id']).fillna(0)
df_all['Count_nuclei'] = df_all['Count_nuclei'].astype(int)
df_all['Count_cyto'] = df_all['Count_cell'] - df_all['Count_nuclei'] # Cyto count

# Ranme columns
df_all.columns = ['Condition', 'FOV', 'PPI', 'Id', 'Cell', 'Nuclei',
       'Cyto']

# Plot descrition
display(df_all.describe())
display(df_all.groupby(['Condition', 'FOV', 'Id']).sum().describe())

,Id,Cell,Nuclei,Cyto
count,10947.000000,10947.000000,10947.000000,10947.000000
mean,205.209829,20.432995,6.469078,13.963917
std,124.172420,31.579238,9.234348,26.709337
min,5.000000,1.000000,0.000000,-47.000000
25%,99.000000,3.000000,0.000000,1.000000
50%,198.000000,10.000000,3.000000,5.000000
75%,298.000000,23.000000,9.000000,15.000000
max,519.000000,706.000000,96.000000,691.000000


C:\Users\thu71\AppData\Local\Temp\ipykernel_78316\2680352089.py:34: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  display(df_all.groupby(['Condition', 'FOV', 'Id']).sum().describe())


,Cell,Nuclei,Cyto
count,1491.000000,1491.000000,1491.000000
mean,150.020121,47.496311,102.523810
std,85.268889,26.813884,85.771626
min,15.000000,0.000000,-131.000000
25%,91.000000,32.000000,45.500000
50%,134.000000,47.000000,87.000000
75%,187.000000,63.000000,138.000000
max,1132.000000,234.000000,1112.000000


In [24]:
# Filter out by maximum number of counts per cell
min_count = 100
max_count = 400

df_all = df_all.groupby(['Condition', 'FOV', 'Id']).filter(lambda x: x['Cell'].sum() > min_count)
df_all = df_all.groupby(['Condition', 'FOV', 'Id']).filter(lambda x: (x['Cell'] < max_count).all())
# df_all = df_all.groupby(['Condition', 'FOV', 'Id']).filter(lambda x: (x['Cyto'] >= 0).all())

# Plot descrition
display(df_all.describe())
display(df_all.groupby(['Condition', 'FOV', 'Id']).sum().describe())

,Id,Cell,Nuclei,Cyto
count,7943.000000,7943.000000,7943.000000,7943.000000
mean,205.982752,23.991313,6.577112,17.414201
std,127.745461,34.790571,9.539739,29.076617
min,8.000000,1.000000,0.000000,-40.000000
25%,93.000000,3.000000,0.000000,2.000000
50%,195.000000,13.000000,3.000000,8.000000
75%,300.000000,26.000000,9.000000,19.000000
max,519.000000,395.000000,96.000000,380.000000


C:\Users\thu71\AppData\Local\Temp\ipykernel_78316\2305364778.py:11: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  display(df_all.groupby(['Condition', 'FOV', 'Id']).sum().describe())


,Cell,Nuclei,Cyto
count,1047.000000,1047.000000,1047.000000
mean,182.008596,49.896848,132.111748
std,75.522975,27.789169,77.818659
min,101.000000,0.000000,-130.000000
25%,128.000000,34.000000,80.500000
50%,161.000000,49.000000,117.000000
75%,211.000000,66.000000,163.000000
max,703.000000,234.000000,703.000000


# ML model

In [25]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import preprocessing, metrics
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split, KFold
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import StandardScaler
import wandb

In [39]:
scaler = StandardScaler()

In [40]:
condition = 'ML'
project_name = f'PLA_9PPI_batch'

In [41]:
models = {
    'Adaboost': AdaBoostClassifier(),
    'DecisionTree': DecisionTreeClassifier(),
    'GradientBoosting' : GradientBoostingClassifier(),
    'NaiveBayes': GaussianNB(),
    'RandomForest': RandomForestClassifier(), 
    'SVM': SVC(probability =True),
    'LogisticRegression':  LogisticRegression(),
    'MLP': MLPClassifier(random_state=1, max_iter=100, hidden_layer_sizes=[16, 16, 16])
}



In [42]:
# Get dataframe seperate into count per cell 
df_count = df_all.pivot(index=['Condition', 'FOV', 'Id'], columns=['PPI']).fillna(0)
df_count_cell = df_count.iloc[:, df_count.columns.get_level_values(0)=='Cell']
df_count_cell = pd.DataFrame(scaler.fit_transform(df_count_cell), index=df_count_cell.index, columns=df_count_cell.columns, )

# Get dataframe seperate into count per nuclei and cyto
df_count_cyto_nuclei = df_count.iloc[:, df_count.columns.get_level_values(0)!='Cell']

# Get Condition into numerical label
le = preprocessing.LabelEncoder()
y = le.fit_transform(df_count.index.get_level_values(0))

In [43]:
df_count_cell

Cell                                         \
PPI                Bim & Tom20 Cyclin D1 & CDK2 Cyclin D1 & CDK2 - re   
Condition  FOV Id                                                       
HCC827Ctrl FW1 13     1.824241         1.951270              1.144715   
               14     3.096289         3.672417              0.040091   
               15    -0.338241        -0.586319             -0.180833   
               17    -0.465446        -1.049705             -0.180833   
               18     0.806602         0.296321             -0.622683   
...                        ...              ...                   ...   
HCC827Osim FW2 336   -0.592650         0.252189              0.040091   
               339    1.315422        -0.056735              1.365640   
               343   -0.465446        -0.299461             -0.180833   
               344   -0.465446        -0.652517              2.470263   
               345   -1.101470        -0.475989              1.144715   

                                                                 \
PPI                Cyclin E & CDK4 Mcl-1 & BAK NF-Kb & p-P90rsk   
Condition  FOV Id                                                 
HCC827Ctrl FW1 13        -0.327841    2.181185         2.520916   
               14        -0.327841    0.029457         3.384905   
               15         0.075034    0.029457        -0.762241   
               17        -0.327841   -0.687786        -1.107836   
               18        -0.327841   -0.687786         0.447343   
...                            ...         ...              ...   
HCC827Osim FW2 336       -0.327841   -0.687786        -0.416646   
               339        0.477909    0.746700         0.447343   
               343       -0.327841   -0.687786        -0.330247   
               344        0.075034   -0.687786        -0.330247   
               345       -0.327841   -0.687786        -0.675842   

                                                                               \
PPI                NF-Kb & p-P90rsk - re P-AKT & mTOR Sox2 & Oct4 TEAD1 & YAP   
Condition  FOV Id                                                               
HCC827Ctrl FW1 13              -0.521190    -0.270094    2.182217   -0.338654   
               14              -0.521190    -0.270094    3.569752    0.759087   
               15              -0.521190     0.117287   -0.592854    0.210217   
               17              -0.521190    -0.270094   -1.082572    0.210217   
               18              -0.521190    -0.270094   -0.021516   -0.338654   
...                                  ...          ...         ...         ...   
HCC827Osim FW2 336             -0.521190     0.117287   -0.266375    0.210217   
               339              1.029055    -0.270094   -0.021516   -0.338654   
               343              1.029055    -0.270094    0.957921   -0.338654   
               344             -0.521190    -0.270094   -0.756093   -0.338654   
               345             -0.521190    -0.270094   -0.511234   -0.338654   

                                  
PPI                p-ERK & c-MYC  
Condition  FOV Id                 
HCC827Ctrl FW1 13       2.785298  
               14       0.481068  
               15       1.294325  
               17       0.481068  
               18      -0.738819  
...                          ...  
HCC827Osim FW2 336     -0.874362  
               339     -0.061104  
               343     -1.009905  
               344     -0.467733  
               345     -1.009905  

[1047 rows x 11 columns]

In [44]:
## Train on P10 an'P10d evaluate on P21
train = 'FW1'
held = 'FW2'

# Run model on cell count
df_train = df_count_cell[df_count.index.get_level_values('FOV') == train]
df_held = df_count_cell[df_count.index.get_level_values('FOV') == held]
X = df_train.values
y = le.transform(df_train.index.get_level_values(0))
X_held = df_held.values
y_held = le.transform(df_held.index.get_level_values(0))

print(len(X), len(y), len(X_held), len(y_held))
print(np.unique(y, return_counts=True), np.unique(y_held, return_counts=True))

460 460 587 587
(array([0, 1]), array([275, 185], dtype=int64)) (array([0, 1]), array([405, 182], dtype=int64))


In [46]:
## Train on P10 an'P10d evaluate on P21
train = 'FW1'
held = 'FW2'

# Run model on cell count
df_train = df_count_cell[df_count_cell.index.get_level_values('FOV') == train]
df_held = df_count_cell[df_count_cell.index.get_level_values('FOV') == held]

X = df_train.values
# X = scaler.fit_transform(X)
y = le.transform(df_train.index.get_level_values(0))

X_held = df_held.values
# X_held = scaler.fit_transform(X_held)
y_held = le.transform(df_held.index.get_level_values(0))

print(len(X), len(y), len(X_held), len(y_held))

# K fold training
kfold = KFold(n_splits = 3, shuffle = True, random_state = 0)
for model_name, model in models.items():
    for k, (train_index, test_index) in enumerate(kfold.split(X)):
        # Split the dataset
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index] 

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_probas = model.predict_proba(X_test)
        y_pred_held = model.predict(X_held)
        y_probas_held = model.predict_proba(X_held)

        run = wandb.init(project=project_name, group=model_name+'_'+train, name=model_name+f'_{train}_{k}')

        accuracy = metrics.accuracy_score(y_test, y_pred)
        accuracy_held = metrics.accuracy_score(y_held, y_pred_held)
        f1 = metrics.f1_score(y_test, y_pred)
        f1_held = metrics.f1_score(y_held, y_pred_held)
        # auc = metrics.roc_auc_score(y_test, y_probas[:, 1])
        # auc_held = metrics.roc_auc_score(y_held, y_probas_held[:, 1])
        auc = metrics.roc_auc_score(y_test, y_pred)
        auc_held = metrics.roc_auc_score(y_held, y_pred_held)
        wandb.log({"accuracy": accuracy, 'accuracy_held': accuracy_held, 'f1':f1, 'f1_held': f1_held, 'auc': auc, 'auc_held': auc_held})
    run.finish()

460 460 587 587


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: thoomas. Use `wandb login --relogin` to force relogin


accuracy,▁
accuracy_held,▁
auc,▁
auc_held,▁
f1,▁
f1_held,▁
accuracy,0.74675
accuracy_held,0.74617
auc,0.73142
auc_held,0.72378
f1,0.67769


accuracy,▁
accuracy_held,▁
auc,▁
auc_held,▁
f1,▁
f1_held,▁
accuracy,0.7451
accuracy_held,0.70187
auc,0.73766
auc_held,0.68109
f1,0.67769


accuracy,▁
accuracy_held,▁
auc,▁
auc_held,▁
f1,▁
f1_held,▁
accuracy,0.72549
accuracy_held,0.74617
auc,0.70952
auc_held,0.74042
f1,0.65


...

c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


accuracy,▁
accuracy_held,▁
auc,▁
auc_held,▁
f1,▁
f1_held,▁
accuracy,0.83117
accuracy_held,0.75809
auc,0.81042
auc_held,0.73242
f1,0.77193


c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


accuracy,▁
accuracy_held,▁
auc,▁
auc_held,▁
f1,▁
f1_held,▁
accuracy,0.77124
accuracy_held,0.73083
auc,0.75535
auc_held,0.7293
f1,0.69565


accuracy,▁
accuracy_held,▁
auc,▁
auc_held,▁
f1,▁
f1_held,▁
accuracy,0.73203
accuracy_held,0.74787
auc,0.70794
auc_held,0.73257
f1,0.63717


In [47]:
## Train on P10 an'P10d evaluate on P21
train = 'FW2'
held = 'FW1'

# Run model on cell count
df_train = df_count_cell[df_count_cell.index.get_level_values('FOV') == train]
df_held = df_count_cell[df_count_cell.index.get_level_values('FOV') == held]

X = df_train.values
# X = scaler.fit_transform(X)
y = le.transform(df_train.index.get_level_values(0))

X_held = df_held.values
# X_held = scaler.fit_transform(X_held)
y_held = le.transform(df_held.index.get_level_values(0))

print(len(X), len(y), len(X_held), len(y_held))

# K fold training
kfold = KFold(n_splits = 3, shuffle = True, random_state = 0)
for model_name, model in models.items():
    for k, (train_index, test_index) in enumerate(kfold.split(X)):
        # Split the dataset
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index] 

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_probas = model.predict_proba(X_test)
        y_pred_held = model.predict(X_held)
        y_probas_held = model.predict_proba(X_held)

        run = wandb.init(project=project_name, group=model_name+'_'+train, name=model_name+f'_{train}_{k}')

        accuracy = metrics.accuracy_score(y_test, y_pred)
        accuracy_held = metrics.accuracy_score(y_held, y_pred_held)
        f1 = metrics.f1_score(y_test, y_pred)
        f1_held = metrics.f1_score(y_held, y_pred_held)
        # auc = metrics.roc_auc_score(y_test, y_probas[:, 1])
        # auc_held = metrics.roc_auc_score(y_held, y_probas_held[:, 1])
        auc = metrics.roc_auc_score(y_test, y_pred)
        auc_held = metrics.roc_auc_score(y_held, y_pred_held)
        wandb.log({"accuracy": accuracy, 'accuracy_held': accuracy_held, 'f1':f1, 'f1_held': f1_held, 'auc': auc, 'auc_held': auc_held})
    run.finish()

587 587 460 460


accuracy,▁
accuracy_held,▁
auc,▁
auc_held,▁
f1,▁
f1_held,▁
accuracy,0.7602
accuracy_held,0.7413
auc,0.6903
auc_held,0.69784
f1,0.56881


accuracy,▁
accuracy_held,▁
auc,▁
auc_held,▁
f1,▁
f1_held,▁
accuracy,0.77041
accuracy_held,0.73478
auc,0.72157
auc_held,0.69061
f1,0.6281


accuracy,▁
accuracy_held,▁
auc,▁
auc_held,▁
f1,▁
f1_held,▁
accuracy,0.7641
accuracy_held,0.7413
auc,0.67838
auc_held,0.70138
f1,0.53061


accuracy,▁
accuracy_held,▁
auc,▁
auc_held,▁
f1,▁
f1_held,▁
accuracy,0.70408
accuracy_held,0.68478
auc,0.66659
auc_held,0.65499
f1,0.54688


...

c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


accuracy,▁
accuracy_held,▁
auc,▁
auc_held,▁
f1,▁
f1_held,▁
accuracy,0.79592
accuracy_held,0.7587
auc,0.73375
auc_held,0.71592
f1,0.63636


c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


accuracy,▁
accuracy_held,▁
auc,▁
auc_held,▁
f1,▁
f1_held,▁
accuracy,0.81122
accuracy_held,0.73913
auc,0.75975
auc_held,0.69514
f1,0.68376


accuracy,▁
accuracy_held,▁
auc,▁
auc_held,▁
f1,▁
f1_held,▁
accuracy,0.86667
accuracy_held,0.76087
auc,0.80793
auc_held,0.71862
f1,0.73469
